# DeCODER Neon testing

## Installs, imports and definitions


In [ ]:
%%capture
!pip install pandas s3fs
!pip install -q oxrdflib
!pip install -q ipysigma
!pip install -q pyoxigraph
!pip install -q pygraphml
!pip install -q pyld

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 28.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.6.0
    Uninstalling fsspec-2023.6.0:
      Successfully uninstalled fsspec-2023.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is no

In [ ]:
# !git clone https://github.com/earthcube/stacIndexer

Cloning into 'stacIndexer'...
remote: Enumerating objects: 508, done.
remote: Counting objects: 100% (508/508), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 508 (delta 397), reused 508 (delta 397), pack-reused 0
Receiving objects: 100% (508/508), 154.63 KiB | 2.09 MiB/s, done.
Resolving deltas: 100% (397/397), done.


In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)  ## remove pandas future warning
from urllib.request import urlopen
from ipysigma import Sigma
import os
import requests
from concurrent.futures import ThreadPoolExecutor
import networkx as nx
from lxml import etree
import io
import pyoxigraph
from rdflib import Graph
from pyld import jsonld
import json
import pandas as pd
import s3fs

from pygraphml import GraphMLParser
from pygraphml import Graph as GraphML

In [2]:
def f2s(file_name):
    with open(file_name, 'r') as file:
        content = file.read()
    return content


## Load to Oxigraph

In [20]:
store = pyoxigraph.Store()  #    store = pyoxigraph.Store(path="./store")
mime_type = "application/n-triples"

In [85]:
# DANGER  clear the store, be sure you run this only if you really want to 
store.clear()

In [86]:
dir = "../data/output"  # from our cloned repo
for f in os.listdir(dir):
  fp = os.path.join(dir, f)

  jld = f2s(fp)
  doc = json.loads(jld)
  normalized = jsonld.normalize(doc, {'algorithm': 'URDNA2015', 'format': 'application/n-quads'})
  # 
  # print("------------------------------")
  # print(normalized)
  # 
  store.load(io.StringIO(normalized), mime_type)

## Simple Early Visualization

In [87]:
rq1 = """	PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <https://schema.org/>

SELECT DISTINCT ?source ?type ?target ?sType ?tType
WHERE {
    ?source a ?sType .
    ?target a ?tType .
    ?source ?type ?target .
}

"""

rq2 = 'SELECT ?s WHERE { ?s ?p ?o } LIMIT 100'

rq3 = """	PREFIX schema: <https://schema.org/>
SELECT ?o
WHERE {
     ?s schema:distribution ?d .
     ?d schema:contentUrl ?o .
 }
"""


In [88]:
qr = list(store.query(rq1))

In [89]:
# print(len(qr))
for r in qr[0:200]:
    if "citation" in r['type'].value:
        print("{} {} {}".format(r['source'].value, r['target'].value, r['type'].value))

In [90]:
#  yeah, I get it...   don't iterate rows...  PR's welcome for this, being bad is too easy!
# for index, row in nodes.iterrows():
#    g.add_node(row['Id'])
g = GraphML()
g.directed = False

for  r in qr:
    n1 = g.add_node(r['source'])
    # n1['name'] = row['name']

    n2 = g.add_node(r['target'])
    # n2['type'] = "Funder"
    # n2['value'] = row['funding.name']

    # n3 = g.add_node(row['ahash'])
    # n3['type'] = "Affiliation"
    # n3['value'] = row['affil']

    e1 = g.add_edge(n1, n2)
    # e2 = g.add_edge(n1, n3)
    # e['predicate'] = row['type']

In [91]:
# For use in Google Colab
# from google.colab import output
# output.enable_custom_widget_manager()

In [92]:
fname = "testGraphML.graphml"
parser = GraphMLParser()
parser.write(g, fname)

In [93]:
g = nx.read_graphml("testGraphML.graphml")

In [94]:
Sigma(
    g,
    node_size=g.degree,
    default_edge_type='curve',
    node_border_color_from='node',
    node_metrics=['louvain'],
    node_color='louvain',
    start_layout=5,
    edge_size=lambda u, v: g.degree(u) + g.degree(v),
    edge_size_range=(0.5, 5),
    label_font='cursive',
    node_label_size=g.degree,
    label_density=0
)

Sigma(nx.MultiGraph with 2,375 nodes and 3,550 edges)

## Simple search and load parquet to Pandas

## Explore Connection to ProtoOKN via Know Where Graph

In [13]:
okn1 = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <https://schema.org/>

SELECT ?s2l13 ?kwgs ?strings2l13 ?curl
WHERE {
  <https://example.org/id/de648e20a4> schema:spatialCoverage  ?sp .
  <https://example.org/id/de648e20a4> schema:distribution ?dist .
  ?dist schema:contentUrl ?curl .
  ?sp schema:geo ?geo .
  ?geo schema:name "s2Level13" .
  ?geo schema:value ?s2l13 .
    BIND (STR(?s2l13) AS ?strings2l13)

}
"""

okn2 = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <https://schema.org/>

SELECT ?kwgs WHERE {

  SERVICE <https://stko-kwg.geog.ucsb.edu/workbench/repositories/KWG> {
            select ?kwgs where {
	        ?kwgs ?kwgp "9291047802178109440" .
        }
	}

}
"""



In [14]:
oknqr = list(store.query(okn1))

In [15]:
df = pd.DataFrame(oknqr)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       8 non-null      object
 1   1       0 non-null      object
 2   2       8 non-null      object
 3   3       8 non-null      object
dtypes: object(4)
memory usage: 388.0+ bytes


In [17]:
df.head(5)

,0,1,2,3
0,"""9291045878032760832""^^<http://www.w3.org/2001...",None,"""9291045878032760832""","""s3://anonymous@bio230014-bucket01/neon4cast-f..."
1,"""9291047802178109440""^^<http://www.w3.org/2001...",None,"""9291047802178109440""","""s3://anonymous@bio230014-bucket01/neon4cast-f..."
2,"""9291047252422295552""^^<http://www.w3.org/2001...",None,"""9291047252422295552""","""s3://anonymous@bio230014-bucket01/neon4cast-f..."
3,"""9291043953887412224""^^<http://www.w3.org/2001...",None,"""9291043953887412224""","""s3://anonymous@bio230014-bucket01/neon4cast-f..."
4,"""9291044503643226112""^^<http://www.w3.org/2001...",None,"""9291044503643226112""","""s3://anonymous@bio230014-bucket01/neon4cast-f..."


These can searched with the SPARQL above or placed into the URL template:

<http://stko-kwg.geog.ucsb.edu/lod/resource/s2.level11.9291047802178109440>  "http://stko-kwg.geog.ucsb.edu/lod/resource/s2.level11.9291052200224620544"

We can also pull and use the S3 link:

s3://anonymous@bio230014-bucket01/neon4cast-forecasts/parquet/aquatics/model_id=VT_Aquaers?endpoint_override=sdsc.osn.xsede.org




In [18]:
import pandas as pd
import s3fs

# S3 URL
s3_url = "s3://anonymous@bio230014-bucket01/neon4cast-forecasts/parquet/aquatics/model_id=GLEON_lm_lag_1day?endpoint_override=sdsc.osn.xsede.org"
frag = "s3://bio230014-bucket01/neon4cast-forecasts/parquet/aquatics"

df = pd.read_parquet(s3_url)

# 
# # Create an S3 filesystem object
# s3 = s3fs.S3FileSystem(anon=True, endpoint_url='https://sdsc.osn.xsede.org')
# 
# # Use the filesystem object to open the Parquet file
# with s3.open(frag, mode='rb') as f:
#     # Load the Parquet file into a pandas DataFrame
#     df = pd.read_parquet(f)
# 
# # Now df contains your data
# print(df)


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1646902 entries, 0 to 1646901
Data columns (total 10 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   target_id           0 non-null        object        
 1   datetime            1646902 non-null  datetime64[us]
 2   parameter           1646902 non-null  object        
 3   variable            1646902 non-null  object        
 4   prediction          1646902 non-null  float64       
 5   family              1646902 non-null  object        
 6   site_id             1646902 non-null  object        
 7   model_id            0 non-null        object        
 8   reference_datetime  1646902 non-null  category      
 9   date                1646902 non-null  category      
dtypes: category(2), datetime64[us](1), float64(1), object(6)
memory usage: 105.2+ MB


In [21]:
df.head()

,target_id,datetime,parameter,variable,prediction,family,site_id,model_id,reference_datetime,date
0,None,2022-10-30,1,temperature,9.336857,sample,LIRO,None,2022-10-29,2022-10-30
1,None,2022-10-30,2,temperature,9.891890,sample,LIRO,None,2022-10-29,2022-10-30
2,None,2022-10-30,3,temperature,10.273416,sample,LIRO,None,2022-10-29,2022-10-30
3,None,2022-10-30,4,temperature,10.719715,sample,LIRO,None,2022-10-29,2022-10-30
4,None,2022-10-30,5,temperature,11.114386,sample,LIRO,None,2022-10-29,2022-10-30
